This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

Author: Alexander Young

### This notebook uses the work flow of Karlsen el al., (2019) to take the files created from 4-Make_SubZone_and_MOR_stats.ipynb and calculate regassing and degassing flux from subduction zone and mor kinematics

##### Input:
file of time-dependent subduction zone kinematics and plate ages e.g. szData_summaryStats_NNRPlateModel.csv

file of  time-dependent mor kinematics e.g. morData_summaryStats_NNRPlateModel.csv

##### Output:
re- and degassing flux data files e.g. degassingFlux_AllTimes_NNRPlateModel.csv

total time-dependent mantle-ocean water exhange file e.g. NNRPlateModel_RD_DeepWaterFlux_AllTimes.csv

##### Citations:
Karlsen, K. S., Conrad, C. P., and Magni, V., 2019, Deep Water Cycling and Sea Level Change Since the Breakup of Pangea: Geochemistry, Geophysics, Geosystems.

Steinberger, B., and Becker, T. W., 2018, A comparison of lithospheric thickness models: Tectonophysics, v. 746, p. 325-338.

In [7]:
import sys
import os
import os.path
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
import scipy.stats as ss
import glob

# allow plots to appear within the notebook
% matplotlib inline

min_time = 0
max_time = 580
time_step = 20

# set model name; should be the same as in 4-Make_SubZone_and_MOR_stats.ipynb
model='NNRPlateModel'

# Set regassing parameterization per Karlsen et al., (2019)
# 0 for RD: regassing dominated
# 1 for LB: longterm balance
Regassing_parameterization=1.

# set working directory and make if it does not exist
workDir="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DeepwaterCycling"
if not os.path.exists(workDir):
    os.makedirs(workDir)

In [8]:
# -- Calculate regassing flux
# read in file of time-dependent subduction zone kinematics and plate ages made using 4-Make_SubZone_and_MOR_stats.ipynb
# e.g. szData_summaryStats_NNRPlateModel.csv
df_AllTimes = pd.read_csv(workDir+'/szData_summaryStats_%s.csv' %(model)) 

# drop segment if convergence rate is less than 2 cm/yr or if plate age is less than zero
df_AllTimes = df_AllTimes[df_AllTimes.conv_rate >= 0.2]
df_AllTimes = df_AllTimes[df_AllTimes.plate_age >= 0.0]

# constrain plate age upper limit to 120 Myr
df_AllTimes['plate_age'] = np.where(df_AllTimes['plate_age'] >= 120.0, 120., df_AllTimes['plate_age'])

############################################################################################################
# Below we calculate the orthogonal migration and convergence rates, which are important to get true estimates of area and volume flux

cr = np.asarray(df_AllTimes['conv_rate'])
co = np.asarray(df_AllTimes['conv_obliq'])

df_AllTimes['ortho_conv_rate'] = pd.Series(cr*np.abs(np.cos(np.radians(co))), index=df_AllTimes.index)

# Convert conv rate to m/sec
def convR(row):
    return ((row['conv_rate']) / 3.154e9)
df_AllTimes['conv_rate_m']=df_AllTimes.apply(convR, axis=1)    

# Convert conv rate to m/sec
def convR(row):
    return ((row['ortho_conv_rate']) / 3.154e9)
df_AllTimes['ortho_conv_rate_m']=df_AllTimes.apply(convR, axis=1)    

# Calculate lithosphere thickness using half space method
def litho_thickness(row):
    return ((10.*np.sqrt(row['plate_age']))*1000.)
df_AllTimes['litho_thickness_m']=df_AllTimes.apply(litho_thickness, axis=1)    

# convert arc length to m
def AngularConversion(row):
    return 2*math.pi*6371000.*((np.asarray(row['arc_length']))/360.)
df_AllTimes['arc_length_m']=df_AllTimes.apply(AngularConversion, axis=1)    

# convert myr to s
def TimeConversion(row):
    return row['plate_age']*3.1536e13
df_AllTimes['plate_age_s']=df_AllTimes.apply(TimeConversion, axis=1)    

# Calculate lithosphere thickness using Steinberger et al., 2018
def litho_thickness_bs18(row):
    return (10. * (np.sqrt(row['plate_age'])) * 1e3)
df_AllTimes['litho_thickness_bs18_m']=df_AllTimes.apply(litho_thickness_bs18, axis=1)    
df_AllTimes.loc[df_AllTimes.litho_thickness_bs18_m>1e5,'litho_thickness_bs18_m'] = 1e5

# Calculate thermal parameter as in Karlsen et al., (2019)
def phi(row):
    return ((row['ortho_conv_rate_m']) * (row['plate_age_s']))/1e4
df_AllTimes['phi']=df_AllTimes.apply(phi, axis=1)    

# Calculate subducting plates relative water retention as in Karlsen et al., (2019)
a = -0.1
b = 0.5
c = 0.0023

def epsilon(row):
    return max(0.0, ((a + b * (1 - math.exp(-c * row['phi'])))))
df_AllTimes['epsilon']=df_AllTimes.apply(epsilon, axis=1)    

# Calculate deep water regassing flux as in Karlsen et al., (2019)
rho = 3200.
if Regassing_parameterization == 0.:
    scenario = 'RD'# regassing dominated RD
    ap = 2.28e-3 
else:
    scenario = 'LB'# Longterm balance LB
    ap = 1.07e-3 

def Regassing_bs18(row):
    return ((ap * row['epsilon'] * rho * row['litho_thickness_bs18_m'] * row['ortho_conv_rate_m'] * row['arc_length_m']) * 3.154e7)
df_AllTimes['R_t_bs18']=df_AllTimes.apply(Regassing_bs18, axis=1)    

In [9]:
# Calculate time-dependent regassing flux
regassing_bs18 = []
total_length = []
average_velocity = []
average_ortho_velocity = []
mean_age = []
epsilon = []
phi = []
mean_thickness =[]

TimeStepList = np.arange(min_time, max_time, time_step)

for TIME in TimeStepList:
    subset = df_AllTimes[(df_AllTimes['reconstruction_time']>=TIME) & (df_AllTimes['reconstruction_time']<(TIME+time_step))]
    regassing_bs18.append(np.nansum(np.asarray(subset.R_t_bs18)))
    total_length.append(np.nansum(np.asarray(subset.arc_length_m)))
    average_velocity.append(subset['conv_rate'].mean())
    average_ortho_velocity.append(subset['ortho_conv_rate'].mean())
    mean_age.append(subset['plate_age'].mean())
    mean_thickness.append(subset['litho_thickness_bs18_m'].mean())

    epsilon.append(subset['epsilon'].mean())
    phi.append(subset['phi'].mean())

regassing_df = pd.DataFrame({'Time':TimeStepList, 'regassing_bs18':regassing_bs18, 'sz_total_length':total_length,
                             'sz_average_velocity':average_velocity, 'average_ortho_velocity':average_ortho_velocity,
                             'mean_age':mean_age, 'mean_thickness':mean_thickness, 'epsilon':epsilon, 'phi':phi})

regassing_df.to_csv(workDir+'/regassingFlux_AllTimes_%s_%s.csv' %(model, scenario))

In [10]:
# -- Calculate degassing flux 
# read in file of time-dependent mor kinematics made using 4-Make_SubZone_and_MOR_stats.ipynb
# e.g. morData_summaryStats_NNRPlateModel.csv
MORData_df = pd.read_csv(workDir+'/morData_summaryStats_%s.csv' %(model)) 

# remove segments where mor velocity is less than 2 cm/yr
MORdf_AllTimes = MORData_df[MORData_df.spreading_rate >= 0.2]

# convert arc length to m
def AngularConversion(row):
    return 2*math.pi*6371000.*((np.asarray(row['arc_length']))/360.)
MORdf_AllTimes['arc_length_m']=MORdf_AllTimes.apply(AngularConversion, axis=1)    

# Convert spreadingrate to m
def spr_rate(row):
    return ((row['spreading_rate']) / 100.)
MORdf_AllTimes['spreading_rate_m']=MORdf_AllTimes.apply(spr_rate, axis=1)    

# calculate degassing as in Karlsen et al., (2019)
def Degassing(row):
    gamma = 3.15e-3
    h = 7000.
    return ((gamma) * (rho) * (h) * (row['spreading_rate_m']) * (row['arc_length_m']))

MORdf_AllTimes['D_t']=MORdf_AllTimes.apply(Degassing, axis=1)

/Users/ajy321/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/ajy321/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ajy321/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [11]:
# Calculate time-dependent degassing flux
degassing = []
total_length = []
average_velocity = []

for TIME in TimeStepList:
    subset1 = MORdf_AllTimes[(MORdf_AllTimes['reconstruction_time']==TIME)]
    degassing.append(math.fsum(subset1.D_t))    
    total_length.append(math.fsum(np.asarray(subset1.arc_length_m)))
    average_velocity.append(subset1['spreading_rate'].mean())

degassing_df = pd.DataFrame({'Time':TimeStepList,'degassing':degassing, 'mor_total_length':total_length,
                            'mor_average_velocity':average_velocity})

def invert(row):
    return row['degassing'] * -1.
degassing_df['degassing']=degassing_df.apply(invert, axis=1)    

degassing_df.to_csv(workDir+'/degassingFlux_AllTimes_%s.csv' %(model))

In [12]:
# -- calculate deep water flux 
# -- combine regassing and degassing dataframes
dfFINAL = pd.merge(degassing_df, regassing_df, on='Time', how='outer')

# convert degassing and regassing data to array
degassing=np.array(dfFINAL['degassing'])
regassing=np.array(dfFINAL['regassing_bs18'])

dt = 20e6 # Myear time step
    
# calculate net water flux and integrate over time
height = (degassing+regassing) / 1000.
mheight = np.zeros(height.shape)
mheight[1:] = [(height[i]+height[i-1])/2.0 for i in range(1,len(height))]
dfFINAL['volume_m3'] = np.cumsum(mheight*dt)

# export to file
dfFINAL.to_csv(workDir+'/%s_%s_DeepWaterFlux_AllTimes.csv' %(model, scenario))
